In [1]:
import torch.nn as nn
import torch
import torch.optim as optim
import torchvision
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter

/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.i

# 모델 생성

In [2]:
class DenseNet121_change_avg(nn.Module):
    def __init__(self):
        super(DenseNet121_change_avg, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True).features
        self.avgpool = nn.AdaptiveAvgPool2d(1)  
        self.relu = nn.ReLU()
        self.mlp = nn.Linear(1024, 6)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.densenet121(x)      
        x = self.relu(x)
        x = self.avgpool(x)
        x_features = x.view(-1, 1024)
        x = self.mlp(x_features)
        x = self.sigmoid(x)
        
        return x, x_features
    
model = DenseNet121_change_avg()

In [3]:
from torchsummary import summary
model = DenseNet121_change_avg()
# summary(model, (3,512,512), device='cpu')

# Transform 전처리기 생성

In [4]:
import torchvision.transforms as T
transforms = T.Compose([T.Resize([256,256]),
                        T.ToTensor()])

# 데이터셋 세팅

In [5]:
from torch.utils.data import Dataset
from PIL import Image

class HmDataset(Dataset):
    def __init__(self, df_path, transforms=None):
        self.df = pd.read_csv(df_path)
        self.transforms = transforms
        
    def __getitem__(self, index):
        hm_meta = self.df.iloc[index]
        filename = hm_meta.filename
        label = torch.from_numpy(hm_meta['epidural':'any'].values.astype(np.float))
        
        img = Image.open('../dataset/kaggle_rsna(only100)/imgs/'+filename+'.png')
        if self.transforms is not None:
            img = self.transforms(img)
        
        return filename, label, img
    
    def __len__(self):
        return len(self.df)

train_dataset = HmDataset(df_path='./train.csv', transforms=transforms)
valid_dataset = HmDataset(df_path='./valid.csv', transforms=transforms)
test_dataset = HmDataset(df_path='./valid.csv', transforms=transforms)

In [6]:
from torch.utils.data import DataLoader
# Hyper Parameter
BATCH_SIZE = 64
EPOCHS = 10

train_loader = DataLoader(train_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True,
                         num_workers=4)
valid_loader = DataLoader(valid_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4)

# 학습

In [7]:
model =  DenseNet121_change_avg()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.cuda()

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

writer = SummaryWriter()

In [8]:
def fit(phase, epoch, model, data_loader, optimizer, criterion, device):
    
    losses = []
    if phase=='Train':
        model.train()
    elif phase=='Valid' or phase=='Test':
        model.eval()
    
    tbar = tqdm(data_loader, position=0, leave=True)
    for data in tbar:
        
        _, target, input_img = data
        target, input_img = target.to(device), input_img.to(device)
        
        optimizer.zero_grad()
        
        predicted_label, predicted_features = model(input_img)
        loss = criterion(predicted_label, target.float())
        
        if phase=='Train':
            loss.backward()
            optimizer.step()
        
#         predicted_label_thresholded = predicted_label>0.5
#         acc = (predicted_label_thresholded==target).sum() # 

        losses.append(loss.item())
        
        tbar.set_description(f'[{phase}]\tEpoch:[{epoch+1}/{EPOCHS}]\tLoss:{loss:.5f}')# '\tAcc:{acc:.2%}')
        
    return losses
        

In [ ]:
from tqdm import tqdm

train_losses = []
valid_losses = []

for epoch in range(EPOCHS):
    train_loss = fit('Train', epoch, model, train_loader, optimizer, criterion, device)
    with torch.no_grad():
        valid_loss = fit('Valid', epoch, model, valid_loader, optimizer, criterion, device)
        
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    

[Train]	Epoch:[2/10]	Loss:0.16601:  45%|████▌     | 20/44 [00:33<00:38,  1.62s/it]

In [85]:
print(train_losses)
print(valid_losses)

12